In [4]:
# urllib のコンソール非表示用
from subprocess import CREATE_NO_WINDOW
# urllib のインポート
import urllib.request, urllib.error
# messagebox のインポート
from tkinter import messagebox
# webdriver のインポート
from selenium import webdriver
# filedialog のインポート
from tkinter import filedialog
# tkinter.tkk のインポート
import tkinter.ttk as ttk
# Path のインポート
from pathlib import Path
# tikinter のインポート
import tkinter as tk
# cv2 のインポート
import cv2
# os のインポート
import os

# ■URL のチェック処理
def checkURL(url):
    try:  
        # GUIから取得したURLのページを開く
        #（開けた＝ページが存在する＝URLが正しい）
        #（開けない→エラー出る→「except」へ）
        f = urllib.request.urlopen(url)
        
        # 開いたのを閉じる
        f.close()
        
        # URLを「start」に返す
        return url
    
    # エラーの場合（ページを開けなかった＝URLが正しくない）
    except:
        # エラーウィンドウ、エラーメッセージ表示
        messagebox.showerror('エラー', 'URLが正しくありません')
        
        # カメラ停止
        capture.release()

# ■カメラ設定の処理
def checkCAM(CameraName):
    
    # GUIで内臓カメラが選択されていた場合
    if CameraName == "内臓カメラ":
        #カメラ引数として「0」に設定
        CameraName = int(0) 
    
    # ↑以外（GUIで外側カメラが選択されていた場合）
    else:
        # カメラ引数として「1」に設定
        CameraName = int(1)

    #「capture」↑で設定したカメラから映像キャプチャ
    capture = cv2.VideoCapture(CameraName)
    
    #「capture」をstartに返す
    return capture



# 分類器パスの確認
def cascade_check(CasPath, caputure):

    # カスケード分類器の設定テキストボックスが空白の時
    if CasPath == "":
        # エラーウィンドウ、エラーメッセージ表示
        messagebox.showerror('エラー', 'カスケードファイルが未入力です')
        
        # カメラを止め処理を終了
        caputure.release()
        sys.exit()
    # カスケード分類器の設定テキストボックスが間違っているとき
    elif os.path.exists(CasPath) == False:
         # エラーウィンドウ、エラーメッセージ表示
        messagebox.showerror('エラー', 'カスケードファイルが見つかりません')
        
        # カメラを止め処理を終了
        caputure.release()
        sys.exit()
    
# ■分類器のパス取得
def dirdialog_clicked():
    
    # 拡張子が「.xml」のファイルのみを表示する
    fTyp = [("", "*.xml")]
    # カレントディレクトリを取得
    current_dir = os.path.abspath(Path().resolve())
    # 上記二つの制限でフォルダを選択するウィンドを作成する
    # dir_path に選択したパスを格納する
    dir_path = filedialog.askopenfilename(filetypes = fTyp, initialdir = current_dir)
    # StringVar 値を設定する
    entry_ws.set(dir_path)
    
# ■ブラウザ立ち上げる処理の関数
def web(url):
    
    # グローバル変数「driver」を宣言
    global driver
    
    # Chromeの操作に「chromedriver.exe」を使う
    driver = webdriver.Chrome(executable_path="C:\chromedriver.exe")
    
    # Chromeの起動、引数でURLを指定（URLはGUIのテキストボックスから）
    driver.get(url)
    
    # Chromeウィンドウを最大化する
    driver.maximize_window()
    

# ■起動時の処理の関数
def start():
    
    # テキストボックスの値を取得して「Cas」に代入
    Cas = dir_entry.get()
    # コンボボックスの値を取得して「CameraName」に代入
    CameraName = CameraBox.get()
    # テキストボックスの値を取得して「urlc」に代入
    urlc = UrlBox.get()
    
    # check 処理
    #「url」に「checkURL関数」から返ってきた値代入
    url = checkURL(urlc)
    #「capture」に「checkCam関数」から返ってきた値代入
    capture = checkCAM(CameraName)
    #「cascade」に「checkCAS関数」から返ってきた値代入
    cascade_check(Cas, capture)
    
    # フラグ作成
    firstloop = True
    
    # カスケード分類器の準備
    cascade = cv2.CascadeClassifier(Cas)
    
    while(capture.isOpened()):
        
        # read()でキャプチャ映像から1コマずつ画像にして「frame」に代入し続ける
        ret,frame = capture.read()
        
        #「gray」に「frame」の画像をグレースケールに変換して代入
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        
        #「cascade」に「checkCAS2関数」の処理代入（顔検出の処理）
        face = cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=3, minSize=(30, 30))
        
        #「Frame」という名前のウィンドウ定義
        cv2.namedWindow("Frame", cv2.WINDOW_NORMAL)
        
        #「Frame」ウィンドウにキャプチャし続けている画像表示
        cv2.imshow("Frame",frame)
        
        # 顔検出処理「face」を動かし続ける
        for i in face:
            #「web関数」にURLを渡してChrome起動
            web(url)
                
            # フラグ「firstloop」を False に切り替え
            firstloop = False
                
        # ブラウザが立ち上がった後の処理
        
        # フラグ「firstloop」を False になったら（顔検出したら）
        if firstloop == False:
            
            # カメラ映像を映しているウィンドウを閉じる
            cv2.destroyWindow("Frame")
            
            # カメラ停止
            capture.release()
            
            # 処理停止
            break
            
        # cv2.waitKeyは()で指定した値（ms）の間キー入力を待つ
        # 今回は1msごとで「q」キー入力を待つ
        #「q」が押されたら
        if cv2.waitKey(1) & 0xFF == ord('q'):
            
            # カメラ映像を映しているウィンドウを閉じる
            cv2.destroyWindow("Frame")
            
            # カメラ停止
            capture.release()
            
            # 処理停止
            break
                   
# ────　Tkinter　────
# メインウィンドウ（名前：tk）の設定
root = tk.Tk()

# タイトル設定
root.title("授業がサボれるアプリ")


# ウィンドウの大きさ指定
# ウィンドウの最大、最小サイズを全く同じにすることで、
# ウィンドウの大きさを変えられないようにしている（デザインがブッ壊れるため）
root.geometry("400x250")

# ウィンドウの最大サイズ指定
root.minsize(width = 400, height = 250)

# ウィンドウの最小サイズ指定
root.maxsize(width = 400, height = 250)

# ラベルの設定
# 引数：ウィンドウ指定、テキスト
label = tk.Label(root, text = "授業がサボれるアプリ")

# ラベルの場所設定
# 横位置指定、縦位置指定、真ん中に合わせて配置
label.place(relx = 0.5,
            y = 35,
            anchor = tk.CENTER)
# ラベルの設定
# 引数：ウィンドウ指定、テキスト
label = tk.Label(root,text = "カスケードファイルパス")

# ラベルの場所設定
# 引数：横位置指定、縦位置指定
label.place(relx = 0.1, y = 60)

# ラベルの設定
# 引数：ウィンドウ指定、テキスト
label = tk.Label(root, text = "URL")
# ラベルの場所設定
# 引数：横位置指定、縦位置指定
label.place(relx = 0.1, y = 100)

# ラベルの設定
# 引数：ウィンドウ指定、テキスト
label = tk.Label(root, text = "カメラ設定")
# ラベルの場所設定
# 引数：横位置指定、縦位置指定
label.place(relx = 0.1, y = 140)



# URLのテキストボックスの設定
UrlBox = tk.Entry(root)

# テキストボックスにデフォルトで入っている文字指定
UrlBox.insert(0, "https://www.google.com/?hl=ja")

# テキストボックスの位置設定
# 引数：横位置指定、幅指定、縦位置指定、真ん中に合わせて配置
UrlBox.place(relx = 0.5,
             relwidth = 0.8,
             y = 130,
             anchor = tk.CENTER)


# コンボボックスの設定
# 選択項目のリスト作成
list1 = ("内臓カメラ", "外部カメラ")

# コンボボックス作成
# 中身をキーボードから入力できないように設定
# 選択項目は「list1」を指定
CameraBox = ttk.Combobox(root,
                         values = list1,
                         state = "readonly")
# デフォルトで「内臓カメラ」を選択状態に
CameraBox.set("内臓カメラ")

CameraBox.insert(0, 0)

# コンボボックスの場所設定
# 引数：横位置指定、幅指定、縦位置指定
CameraBox.place(relx = 0.1,
                relwidth = 0.2,
                y = 160)

# 分類器パス取得
entry_ws = tk.StringVar()
# entry_ws の値が変更されたらその都度テキストボックスの中身を変更
dir_entry = ttk.Entry(root,
                      textvariable = entry_ws)

# テキストボックスの場所設定
# 引数：横位置指定、幅指定、縦位置指定、横位置指定、真ん中に合わせて配置
dir_entry.place(relx = 0.45,
                relwidth = 0.65,
                y = 90,
                x = -10,
                anchor = tk.CENTER)

# 参照ボタンの設定
# 引数：テキストの設定、押されると dirdialog_clicked 関数を動かす、ボタンの色、文字の色の設定
dir_button = ttk.Button(root,
                        text="参照",
                        command = dirdialog_clicked)

# ボタンの場所設定
# 引数：横位置指定、幅指定、縦位置指定、横位置指定、真ん中に合わせて配置
dir_button.place(relx = 0.2,
                 width = 50,
                 y = 90,
                 x = 255,
                 anchor = tk.CENTER)

# 起動ボタンの設定
# 引数：テキストの設定、押されるとstart関数を動かす、ボタンの色、文字の色の設定
button1 = tk.Button(root, text = "起動",
                    command = start,
                    bg = "#F5F5F5",
                    fg = "black")


# ボタンの場所設定
# 引数：横位置指定、幅指定、縦位置指定、真ん中に合わせて配置
button1.place(relx = 0.5,
              relwidth = 0.8,
              y = 210,
              anchor = tk.CENTER)


# プログラムを無限ループさせており、アプリを意図的に終了させない限りアプリを待機させる役割
root.mainloop()

<ipython-input-4-97dd1a8ab526>:100: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path="C:\chromedriver.exe")
